# Import libraries and Create Reference Data

# %% [markdown]
# # Generate Synthetic Financial Test Data (Jupyter Version)
# 
# This notebook generates **Counterparties.csv** and **Transactions.csv** that match the
# updated requirements for your Starburst / NetworkX demos.  
# *Default: 50 counterparties, 200 transactions (date range 2024‑01‑01 → 2025‑03‑31).*
# 
# You can run the whole notebook top‑to‑bottom or tweak the parameters in the first
# cell and rerun just the generation steps.
# 
# --

In [7]:

# %%
# Parameters – tweak as needed
NUM_COUNTERPARTIES = 50
NUM_TRANSACTIONS = 200
TRADE_START = "2024-01-01"
TRADE_END = "2025-03-31"
OUTPUT_DIR = "./data"  # folder to write CSVs

# %%
# Imports & helper functions
import random, string
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
from pandas.tseries.offsets import BusinessDay

In [8]:

def rand_cusip() -> str:
    """Return a 9‑character alphanumeric CUSIP starting with '9'."""
    return "9" + "".join(random.choices(string.ascii_uppercase + string.digits, k=8))


def rand_biz_day(start: datetime, end: datetime) -> pd.Timestamp:
    """Random business day between *start* and *end* (inclusive)."""
    bdays = pd.bdate_range(start=start, end=end, freq="C")
    return pd.Timestamp(np.random.choice(bdays))


# %% [markdown]
# ## Generate Counterparties

In [9]:
# %%
types = ["Bank", "Broker‑Dealer", "Hedge Fund", "Asset Manager"]
subtypes = {
    "Bank": ["Global", "Regional", "Community"],
    "Broker‑Dealer": ["Primary", "Retail"],
    "Hedge Fund": ["Macro", "Long/Short", "Quant"],
    "Asset Manager": ["Pension", "Mutual Fund"],
}
domiciles = ["US", "UK", "CA", "DE", "JP"]
members_flag = ["Yes", "No"]

In [ ]:


def make_counterparties(n: int) -> pd.DataFrame:
    rows = []
    for i in range(n):
        cp_id = f"CP{str(i + 1).zfill(3)}"
        ctype = random.choice(types)
        rows.append({
            "id": cp_id,
            "name": f"Party {i + 1}",
            "type": ctype,
            "subtype": random.choice(subtypes[ctype]),
            "domicile": random.choice(domiciles),
            "ficcmember": random.choice(members_flag),
        })
    return pd.DataFrame(rows)

In [11]:
counterparties_df = make_counterparties(NUM_COUNTERPARTIES)
counterparties_df.head()


,id,name,type,subtype,domicile,member
0,CP001,Party 1,Bank,Global,CA,No
1,CP002,Party 2,Bank,Community,UK,No
2,CP003,Party 3,Broker‑Dealer,Primary,CA,Yes
3,CP004,Party 4,Bank,Global,CA,No
4,CP005,Party 5,Asset Manager,Pension,UK,No


# %% [markdown]
# ## Generate Transactions

In [12]:
# %%
collateral_types = [
    "Government Bond",
    "Corporate Bond",
    "Equity",
    "Treasury Bill",
    "Agency MBS",
]
clearinghouses = ["FICC", "LCH", "ICE"]


In [13]:
def make_transactions(n: int, cp_ids, start: str, end: str) -> pd.DataFrame:
    rows = []
    for i in range(n):
        trade_dt = rand_biz_day(pd.Timestamp(start), pd.Timestamp(end))
        settle_dt = trade_dt + BusinessDay(1)
        lender, borrower = random.sample(cp_ids, 2)
        rows.append({
            "id": f"T{str(i + 1).zfill(4)}",
            "tradeDate": trade_dt.strftime("%Y-%m-%d"),
            "settlementDate": settle_dt.strftime("%Y-%m-%d"),
            "maturityDate": settle_dt.strftime("%Y-%m-%d"),
            "cashAmount": round(random.uniform(1_000_000, 10_000_000), 2),
            "repoRate": round(random.uniform(0.01, 0.10), 4),
            "collateralType": random.choice(collateral_types),
            "CUSIP": rand_cusip(),
            "clearinghouse": random.choice(clearinghouses),
            "cashlender": lender,
            "cashborrower": borrower,
        })
    return pd.DataFrame(rows)

transactions_df = make_transactions(
    NUM_TRANSACTIONS, counterparties_df["id"].tolist(), TRADE_START, TRADE_END
)
transactions_df.head()

,id,tradeDate,settlementDate,maturityDate,cashAmount,repoRate,collateralType,CUSIP,clearinghouse,cashlender,cashborrower
0,T0001,2024-10-31,2024-11-01,2024-11-01,5015701.38,0.0311,Agency MBS,9JADQEHY0,ICE,CP037,CP003
1,T0002,2024-07-15,2024-07-16,2024-07-16,3858548.29,0.0889,Agency MBS,9SFHS8P6J,FICC,CP050,CP020
2,T0003,2024-10-18,2024-10-21,2024-10-21,2439639.15,0.0208,Agency MBS,9H255HDOS,LCH,CP026,CP012
3,T0004,2024-05-30,2024-05-31,2024-05-31,5044036.04,0.0974,Government Bond,96EXPKJLA,LCH,CP001,CP019
4,T0005,2024-03-15,2024-03-18,2024-03-18,4925484.13,0.0554,Equity,98LMHNL0Z,FICC,CP024,CP003


# %% [markdown]
# ## Save CSVs

In [14]:
# %%
output_dir = Path(OUTPUT_DIR)
output_dir.mkdir(parents=True, exist_ok=True)

cp_path = output_dir / "Counterparties.csv"
tx_path = output_dir / "Transactions.csv"

counterparties_df.to_csv(cp_path, index=False)
transactions_df.to_csv(tx_path, index=False)

print(f"✅ Wrote {cp_path}\n✅ Wrote {tx_path}")

✅ Wrote data\Counterparties.csv
✅ Wrote data\Transactions.csv


# %% [markdown]
# ### Quick sanity checks


In [15]:

# %% [markdown]
# ### Quick sanity checks

# %%
print("Counterparty sample:\n", counterparties_df.sample(5).to_string(index=False))
print("\nTransaction sample:\n", transactions_df.sample(5).to_string(index=False))

Counterparty sample:
    id     name          type     subtype domicile member
CP027 Party 27 Broker‑Dealer      Retail       DE    Yes
CP013 Party 13 Broker‑Dealer     Primary       DE    Yes
CP040 Party 40 Broker‑Dealer      Retail       DE    Yes
CP036 Party 36 Asset Manager Mutual Fund       CA     No
CP006  Party 6    Hedge Fund       Macro       JP     No

Transaction sample:
    id  tradeDate settlementDate maturityDate  cashAmount  repoRate collateralType     CUSIP clearinghouse cashlender cashborrower
T0112 2024-12-06     2024-12-09   2024-12-09  9980188.15    0.0655  Treasury Bill 96LG3LW2G           ICE      CP047        CP004
T0148 2024-01-11     2024-01-12   2024-01-12  3463550.37    0.0981  Treasury Bill 97K6UE2Y7           ICE      CP032        CP001
T0126 2024-08-08     2024-08-09   2024-08-09  1911646.48    0.0159 Corporate Bond 9MY8MMWJ6           ICE      CP028        CP042
T0179 2024-11-22     2024-11-25   2024-11-25  2370980.66    0.0467  Treasury Bill 9T8WNPPSR   

In [1]:
import numpy as np
import pandas as pd 
import datetime
import random

# Set variables
credit_ratings = ['AAA', 'AA', 'A', 'BBB']
collateral_types = ['Government Bond', 'Corporate Bond', 'Equity', 'Treasury Bill', 'Agency MBS']
repo_types = ['Repo', 'Reverse Repo', 'Open Repo', 'Term Repo', 'Overnight Reverse Repo', 'Tri-Party Repo', 'Standing Repo Facility']

haircut_ranges = {
    'Government Bond': (0.01, 0.03), # 1-3%
    'Corporate Bond': (0.03, 0.10), # 3-10%
    'Equity': (0.05, 0.20), # 5-20%  
    'Treasury Bill': (0.005, 0.015), # .5-1.5%
    'Agency MBS': (0.02, 0.05) # 2-5%
}


In [2]:
print(credit_ratings)
print(collateral_types)
print(repo_types)

['AAA', 'AA', 'A', 'BBB']
['Government Bond', 'Corporate Bond', 'Equity', 'Treasury Bill', 'Agency MBS']
['Repo', 'Reverse Repo', 'Open Repo', 'Term Repo', 'Overnight Reverse Repo', 'Tri-Party Repo', 'Standing Repo Facility']


# Create Counterparties Data Frame

In [3]:
num_counterparties = 50

counterparties = pd.DataFrame({
    'CounterpartyID': range(1, num_counterparties+1),
    'CounterpartyName': [f'Party {i}' for i in range(1, num_counterparties + 1)],
    'CreditRating': [random.choice(credit_ratings) for _ in range(num_counterparties)]
})

print(counterparties)


    CounterpartyID CounterpartyName CreditRating
0                1          Party 1            A
1                2          Party 2            A
2                3          Party 3          AAA
3                4          Party 4           AA
4                5          Party 5           AA
5                6          Party 6            A
6                7          Party 7          AAA
7                8          Party 8          AAA
8                9          Party 9            A
9               10         Party 10           AA
10              11         Party 11           AA
11              12         Party 12          BBB
12              13         Party 13          BBB
13              14         Party 14            A
14              15         Party 15            A
15              16         Party 16          BBB
16              17         Party 17            A
17              18         Party 18          AAA
18              19         Party 19          BBB
19              20  

# Create Collateral

In [4]:
num_collateral = 200
collateral = pd.DataFrame({
    'CollateralID': range(1, num_collateral+1),
    'CollateralType': [random.choice(collateral_types) for _ in range(num_collateral)],
    'MarketValue': [random.uniform(1000000, 50000000) for _ in range(num_collateral)]
})

collateral['Haircut'] = collateral.apply(lambda row: random.uniform(*haircut_ranges[row['CollateralType']]), axis=1)

# GENERATE TRANSACTIONS

In [5]:
import pandas as pd
import random
from datetime import datetime, timedelta
import string

# Define the date range for start dates
start_date_range = datetime(2024, 1, 1)
end_date_range = datetime(2025, 3, 31)
days_between = (end_date_range - start_date_range).days

def generate_transaction():
    random_days = random.randint(0, days_between)
    start_date = start_date_range + timedelta(days=random_days)
    maturity_date = start_date + timedelta(days=1)  # Overnight repo

    transaction_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8))

    return {
        "ID": transaction_id,
        "startDate": start_date.strftime("%Y-%m-%d"),
        "maturityDate": maturity_date.strftime("%Y-%m-%d"),
        "cashAmount": round(random.uniform(1_000_000, 10_000_000), 2),
        "repoType": random.choice(["Tri-Party", "Bilateral"]),
        "repoRate": round(random.uniform(0.01, 0.10), 4),
        "InitialMargin": round(random.uniform(0.01, 0.03), 4)
    }

# Generate 500 transactions
transactions = [generate_transaction() for _ in range(500)]

# Convert to DataFrame for easier viewing or exporting
df_transactions = pd.DataFrame(transactions)

# Display the first few rows
df_transactions.head()


,ID,startDate,maturityDate,cashAmount,repoType,repoRate,InitialMargin
0,VS25B1N9,2024-10-16,2024-10-17,5634084.24,Bilateral,0.0679,0.0284
1,3PE0B51L,2025-02-16,2025-02-17,6195004.49,Bilateral,0.0923,0.0165
2,OXB9X70Z,2024-04-11,2024-04-12,2038683.14,Tri-Party,0.0387,0.0284
3,JOEOE24E,2025-03-29,2025-03-30,3885165.92,Tri-Party,0.0565,0.0177
4,0J0EHEOL,2024-04-26,2024-04-27,7158036.80,Bilateral,0.0712,0.0121


# Print Counterparties and Transactions Test Data

In [ ]:
print("Counterparties:\n", counterparties.head())
# print("\nCollateral:\n", collateral.head())
print("\Repo Transactions:\n", df_transactions.head())

counterparties.to_csv('./data/counterparties.csv', index=False)
# collateral.to_csv('collateral.csv', index=False)
df_transactions.to_csv('/data/transactions.csv', index=False)

Counterparties:
    CounterpartyID CounterpartyName CreditRating
0               1          Party 1            A
1               2          Party 2            A
2               3          Party 3          AAA
3               4          Party 4           AA
4               5          Party 5           AA

Repo Transactions:
          ID   startDate maturityDate  cashAmount   repoType  repoRate  \
0  VS25B1N9  2024-10-16   2024-10-17  5634084.24  Bilateral    0.0679   
1  3PE0B51L  2025-02-16   2025-02-17  6195004.49  Bilateral    0.0923   
2  OXB9X70Z  2024-04-11   2024-04-12  2038683.14  Tri-Party    0.0387   
3  JOEOE24E  2025-03-29   2025-03-30  3885165.92  Tri-Party    0.0565   
4  0J0EHEOL  2024-04-26   2024-04-27  7158036.80  Bilateral    0.0712   

   InitialMargin  
0         0.0284  
1         0.0165  
2         0.0284  
3         0.0177  
4         0.0121  


## Install Neo4j library

In [7]:
pip install neo4j


  Using cached neo4j-5.28.1-py3-none-any.whl.metadata (5.9 kB)
Using cached neo4j-5.28.1-py3-none-any.whl (312 kB)
Note: you may need to restart the kernel to use updated packages.


## Make a connection to the Neo4j Database. Create two dataframes to store counterparties and transactions from the csvs

In [8]:
from neo4j import GraphDatabase
import pandas as pd

# Load your CSVs (make sure they're in the same directory or update path)
df_counterparties = pd.read_csv("counterparties.csv")
df_transactions = pd.read_csv("transactions.csv")

# Connection credentials
NEO4J_URI = "neo4j://ec2-3-210-231-126.compute-1.amazonaws.com:7687"  # Update this
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "neo4j_enterprise!"  # Replace with actual password

# Set up the driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))


## Load Counteries and Transactions Data into the Neo4j Cluster

In [9]:
def load_counterparties(tx, row):
    query = """
    MERGE (c:Counterparty {id: $CounterpartyID})
    SET c.name = $CounterpartyName,
        c.rating = $CreditRating
    """
    tx.run(query, **row)

def load_transactions(tx, row):
    query = """
    MERGE (t:Transaction {id: $ID})
    SET t.startDate = $startDate,
        t.maturityDate = $maturityDate,
        t.cashAmount = $cashAmount,
        t.repoType = $repoType,
        t.repoRate = $repoRate,
        t.InitialMargin = $InitialMargin
    """
    tx.run(query, **row)


## Write the data into Neo4j

In [10]:
import pandas as pd

# Load the CSVs
df_counterparties = pd.read_csv("counterparties.csv")
df_transactions = pd.read_csv("transactions.csv")

# Write data into Neo4j
with driver.session() as session:
    for _, row in df_counterparties.iterrows():
        session.write_transaction(load_counterparties, row.to_dict())

    for _, row in df_transactions.iterrows():
        session.write_transaction(load_transactions, row.to_dict())


D:\Users\tony.yang\AppData\Local\Temp\1\ipykernel_2288\3999102408.py:10: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(load_counterparties, row.to_dict())
D:\Users\tony.yang\AppData\Local\Temp\1\ipykernel_2288\3999102408.py:13: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(load_transactions, row.to_dict())


## Define the relationships between Counterparty to Transactions
### For this initial set of relationships for a single transaction depending on the repo type (Tri-Party, Bilateral) the following relationship is defined.

### For a given single <b>Tri-Party</b> repo type transaction, there will be 3 unique counter parties associated with the transaction.  Each counterparty will either be a <b>Lender, Borrower, or Intermediary</b>

### For a given single <b>Bilateral</b> repo type transaction, there will be 2 unique counter parties associated with the transaction.  Each counterparty will either be a <b>Lender or Borrower </b>

 

In [11]:
import random

# All available counterparty IDs
counterparty_ids = df_counterparties["CounterpartyID"].tolist()

# Role sets for each transaction type
tri_party_roles = ["lends", "borrows", "intermediates"]
bilateral_roles = ["lends", "borrows"]

# Clear existing relationships first if needed
def clear_existing_relationships(tx):
    tx.run("""
    MATCH (c:Counterparty)-[r]->(t:Transaction)
    DELETE r
    """)

# Relationship creation logic based on repo type
def link_transaction_with_roles(tx, transaction_id, repo_type, counterparties_roles):
    for counterparty_id, role in counterparties_roles:
        query = f"""
        MATCH (c:Counterparty {{id: $counterparty_id}})
        MATCH (t:Transaction {{id: $transaction_id}})
        MERGE (c)-[r:{role.upper()}]->(t)
        """
        tx.run(query, counterparty_id=counterparty_id, transaction_id=transaction_id)

# Apply the logic
with driver.session() as session:
    # Optional: Clear previous relationships
    session.write_transaction(clear_existing_relationships)

    used_assignments = set()  # to track used (transaction_id, counterparty_id) pairs

    for _, row in df_transactions.iterrows():
        tx_id = row["ID"]
        repo_type = row["repoType"]
        
        # Ensure unique counterparties per transaction
        if repo_type == "Tri-Party":
            selected_roles = tri_party_roles
        elif repo_type == "Bilateral":
            selected_roles = bilateral_roles
        else:
            continue  # Skip unknown repo types
        
        # Get N unique counterparties for the transaction
        selected_counterparties = random.sample(counterparty_ids, len(selected_roles))
        
        # Assign roles to those counterparties
        counterparties_roles = list(zip(selected_counterparties, selected_roles))
        
        # Ensure uniqueness across transaction/role/counterparty
        if all((tx_id, cp_id) not in used_assignments for cp_id, _ in counterparties_roles):
            # Register assignments to prevent duplicates
            for cp_id, _ in counterparties_roles:
                used_assignments.add((tx_id, cp_id))
            
            # Write to Neo4j
            session.write_transaction(link_transaction_with_roles, tx_id, repo_type, counterparties_roles)


D:\Users\tony.yang\AppData\Local\Temp\1\ipykernel_2288\2925690090.py:30: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(clear_existing_relationships)
D:\Users\tony.yang\AppData\Local\Temp\1\ipykernel_2288\2925690090.py:59: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(link_transaction_with_roles, tx_id, repo_type, counterparties_roles)
